In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\vellam1\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
# dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
import joblib

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
df.shape

(6991, 41)

# Select your features (columns)

In [6]:
# Set features. This will also be used as your x values, assume all features are potentially relevant with the exception of koi_disposition which will be y
X = df.drop("koi_disposition", 1)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [7]:
y = df["koi_disposition"]
# Split data into training and testing groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state=42, stratify=y)

In [8]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
3570,0,1,0,0,76.819855,0.000004,-0.000004,188.12566,0.00004,-0.00004,...,-152,4.590,0.036,-0.153,0.774,0.182,-0.061,282.00089,44.960770,14.063
6410,0,0,1,1,2.178142,0.000016,-0.000016,132.02946,0.00709,-0.00709,...,-138,4.506,0.070,-0.070,0.840,0.081,-0.081,287.97546,46.289631,15.458
6447,0,0,0,0,21.980467,0.000452,-0.000452,150.72800,0.01800,-0.01800,...,-149,4.418,0.139,-0.186,0.936,0.239,-0.147,289.82422,51.552029,13.852
4528,0,0,0,0,143.163697,0.002400,-0.002400,243.71330,0.01170,-0.01170,...,-254,3.050,0.030,-0.031,7.653,0.334,-2.023,297.78833,40.130909,12.644
3955,0,1,0,0,1.471060,0.000001,-0.000001,132.27522,0.00022,-0.00022,...,-379,3.806,0.425,-0.100,2.911,0.588,-1.371,295.74377,50.830921,15.009


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [9]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [10]:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=random%20forest#sklearn.ensemble.RandomForestClassifier
RFC = RandomForestClassifier(n_estimators = 500)
RFC.fit(X_train_scaled, y_train)

RFC_train_score = round(RFC.score(X_train_scaled, y_train)*100,2)
RFC_test_score = round(RFC.score(X_test_scaled, y_test)*100,2)

print(f"Training Data Score: {RFC_train_score}%")
print(f"Testing Data Score: {RFC_test_score}%")

Training Data Score: 100.0%
Testing Data Score: 88.43%


# Use sklearn RFECV to determine the optimal number of variables
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html
https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html#sphx-glr-auto-examples-feature-selection-plot-rfe-with-cross-validation-py

In [13]:
X_features = X.columns.tolist()
X_features

['koi_fpflag_nt',
 'koi_fpflag_ss',
 'koi_fpflag_co',
 'koi_fpflag_ec',
 'koi_period',
 'koi_period_err1',
 'koi_period_err2',
 'koi_time0bk',
 'koi_time0bk_err1',
 'koi_time0bk_err2',
 'koi_impact',
 'koi_impact_err1',
 'koi_impact_err2',
 'koi_duration',
 'koi_duration_err1',
 'koi_duration_err2',
 'koi_depth',
 'koi_depth_err1',
 'koi_depth_err2',
 'koi_prad',
 'koi_prad_err1',
 'koi_prad_err2',
 'koi_teq',
 'koi_insol',
 'koi_insol_err1',
 'koi_insol_err2',
 'koi_model_snr',
 'koi_tce_plnt_num',
 'koi_steff',
 'koi_steff_err1',
 'koi_steff_err2',
 'koi_slogg',
 'koi_slogg_err1',
 'koi_slogg_err2',
 'koi_srad',
 'koi_srad_err1',
 'koi_srad_err2',
 'ra',
 'dec',
 'koi_kepmag']

In [18]:
# there are 28 features of the original 40 which should be kept in the model. We now need to determine which ones these are, create a new df and remodel
# https://stackoverflow.com/questions/24123498/recursive-feature-elimination-on-random-forest-using-scikit-learn

RFECV_features = sorted(zip(RFC.feature_importances_, X_features), reverse = True)
# RFECV_features
RFECV_df = pd.DataFrame(RFECV_features, columns = ["Score", "Feature"])
RFECV_df.set_index("Feature", inplace = True)
RFECV_df

,Score
Feature,
koi_fpflag_co,0.107013
koi_fpflag_nt,0.087507
koi_fpflag_ss,0.061973
koi_model_snr,0.052260
koi_prad,0.051016
koi_steff_err1,0.037876
koi_prad_err2,0.035672
koi_prad_err1,0.035479
koi_fpflag_ec,0.034217


Through RFECV, there is a gradual reduction in importance of features from a score of 0.11. As the regression model predicted the use of 28 variables, then we will include variables up and a score of 0.01 in this mode.

In [19]:
# remove features with ranking > 28, i.e. keep only variables with ranking = 1
RFECV_vars = []
for ea in RFECV_features:
    if ea[0] >= 0.01:
        RFECV_vars.append(ea[1])
        
RFECV_vars

['koi_fpflag_co',
 'koi_fpflag_nt',
 'koi_fpflag_ss',
 'koi_model_snr',
 'koi_prad',
 'koi_steff_err1',
 'koi_prad_err2',
 'koi_prad_err1',
 'koi_fpflag_ec',
 'koi_steff_err2',
 'koi_duration_err1',
 'koi_duration_err2',
 'koi_duration',
 'koi_time0bk_err1',
 'koi_time0bk_err2',
 'koi_depth',
 'koi_period',
 'koi_period_err2',
 'koi_period_err1',
 'koi_impact',
 'koi_insol_err1',
 'koi_time0bk',
 'ra',
 'koi_depth_err2',
 'koi_depth_err1',
 'koi_teq',
 'koi_srad_err1',
 'koi_slogg_err2',
 'koi_insol',
 'dec',
 'koi_impact_err1',
 'koi_kepmag',
 'koi_insol_err2',
 'koi_impact_err2',
 'koi_srad']

In [20]:
# use new data for models:
X_train_RFECV = X_train[RFECV_vars]
X_test_RFECV = X_test[RFECV_vars]

# X_train_RFECV

X_scaler = MinMaxScaler().fit(X_train_RFECV)
X_train_scaled = X_scaler.transform(X_train_RFECV)
X_test_scaled = X_scaler.transform(X_test_RFECV)

In [21]:
# create a new model using RFECV determined variables
RFC_RFECV = RandomForestClassifier(n_estimators = 500)
RFC_RFECV.fit(X_train_scaled, y_train)

RFC_RFECV_train_score = round(RFC_RFECV.score(X_train_scaled, y_train)*100,2)
RFC_RFECV_test_score = round(RFC_RFECV.score(X_test_scaled, y_test)*100,2)

print(f"Training Data Score: {RFC_RFECV_train_score}%")
print(f"Testing Data Score: {RFC_RFECV_test_score}%")

Training Data Score: 100.0%
Testing Data Score: 88.58%


# Original RFC of that with RFECV determined variables?
Compare the original RFC model to that using RFECV determined variables to see if score has improved

In [22]:
print(f"Training Data Score: {RFC_train_score}%")
print(f"Testing Data Score: {RFC_test_score}%")

print(f"Training Data Score: {RFC_RFECV_train_score}%")
print(f"Testing Data Score: {RFC_RFECV_test_score}%")

Training Data Score: 100.0%
Testing Data Score: 88.43%
Training Data Score: 100.0%
Testing Data Score: 88.58%


# Improvement?
Marginal improvement in the model score when the RFECV determined variables are isolated.

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [23]:
# Create the GridSearchCV model
RFC_GSCV = RandomForestClassifier(random_state = 42)

# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

param_grid = {
    'n_estimators': [200, 600, 1200, 1400],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [14, 15, 16, 17, 18, None]
}

grid = GridSearchCV(RFC_GSCV, param_grid, cv=5, verbose=0, n_jobs=-1)

# Train the model with GridSearch
_ = grid.fit(X_train_scaled, y_train)

In [18]:
# Train the model with GridSearch
C = grid.best_params_['C']
penalty = grid.best_params_['penalty']

logistic_reg_model_GRID = LogisticRegression(solver = 'newton-cg', multi_class = 'auto', C=C, penalty = penalty)
logistic_reg_model_GRID.fit(X_train_scaled, y_train)

logistic_reg_model_GRID_train_score = round(logistic_reg_model_GRID.score(X_train_scaled, y_train)*100,2)
logistic_reg_model_GRID_test_score = round(logistic_reg_model_GRID.score(X_test_scaled, y_test)*100,2)

print(f"Training Data Score: {logistic_reg_model_GRID_train_score}%")
print(f"Testing Data Score: {logistic_reg_model_GRID_test_score}%")

Training Data Score: 89.27%
Testing Data Score: 87.88%


# Best Model?
GridSearchCV returns most accurate model with 87.9% accuracy as compared to 84.3% each for the original regression model and that using RFECV.

# Model Predictions

In [19]:
predictions = logistic_reg_model_GRID.predict(X_test_scaled)
classifications = y_test.unique().tolist()

prediction_actual = {"Actual": y_test, "Predictions": predictions}

predictions_df = pd.DataFrame(prediction_actual)
predictions_df = predictions_df.set_index("Actual").reset_index()
predictions_df.head()

,Actual,Predictions
0,CANDIDATE,CANDIDATE
1,FALSE POSITIVE,FALSE POSITIVE
2,FALSE POSITIVE,FALSE POSITIVE
3,CANDIDATE,CANDIDATE
4,FALSE POSITIVE,FALSE POSITIVE


In [20]:
# Save Evaluations of each model predictions to a CSV
model_evaluations = {"Model": ["Logistic Regression Base", "Logistic Regression (RFECV)", "Logistic Regression (GSCV)"], "Accuracy (%)": [logistic_reg_model_test_score, logistic_reg_model_RFECV_test_score, logistic_reg_model_GRID_test_score]}

eval_df = pd.DataFrame(model_evaluations)
# eval_df
eval_df = eval_df.set_index("Model")

eval_df.to_csv('../RegressionModel_Evaluations.csv')
eval_df

,Accuracy (%)
Model,
Logistic Regression Base,84.31
Logistic Regression (RFECV),84.29
Logistic Regression (GSCV),87.88


# Save the Model

In [21]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = '../RegressionModel.sav'
joblib.dump(logistic_reg_model_GRID, filename)

['../RegressionModel.sav']